This notebook contains the steps of loading in, cleaning, and preprocessing my data for the final project

# Research Questions:
## 1. Descriptives: 
- Were the purchases of jewelry from this retailer at their peak in May of 2011, considering the historical event of Kate and William’s wedding? 
- How does the purchasing of jewelry compare to other types of products,(meaning, per customer what types of products are they buying with jewelry, or are they buying exclusively jewelry, etc.)?
## 2. Main Question:
- Does the price of purchased jewelry predict what country the customer resides in? (target = country)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk 
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [2]:
infile = "Online Retail.xlsx"

df = pd.read_excel(infile)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
# check the measurements of the data
print('Number of Rows: {}'.format(df.shape[0]))
print('Number of Columns: {}'.format(df.shape[1]))
print(f'Are there missing values:\n{df.isna().sum()}')

Number of Rows: 541909
Number of Columns: 8
Are there missing values:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [4]:
# check the data types of the data, so that we know how best to preprocess them
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

We have to make some new columns. The first new column I want to make is the UnitPrice converted to US
dollars, because my audience is American and UnitPrice is currently in sterling (not as common of a currency as euros). Price is also a critical component of my research question. I want the new UnitPrice column to replace the old UnitPrice values. Conversion rates of currencies are always changing, so this rate may not be accurate in the future. 

In [5]:
# Conversion rate taken from Forbes as of April 2024:
# https://www.forbes.com/advisor/money-transfer/currency-converter/gbp-usd/

# 1 GDP = 1.24 USD
# we will multiply each observation's Unit Price value by 1.24

df['UnitPrice'] = df['UnitPrice'].apply(lambda x: x*1.24)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3.1620,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3.4100,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom


In [6]:
# we need to convert our target class 'Country' to 1s and 0s for binary classification

# checking what countries are in this dataframe
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

I want to make a new column called "Target Class" where UK is coded as 1 and all other countries coded as 0. It's important to note here though that it may be interesting to examine grouping countries by their continent to do something other than binary classification, but that goes beyond the scope of my project.

In [7]:
df['Target'] = (df['Country'] == 'United Kingdom').astype(int)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Target
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3.1620,17850.0,United Kingdom,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3.4100,17850.0,United Kingdom,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom,1


In [8]:
# check that other countries got coded as 0
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Target
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,1.054,12680.0,France,0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.604,12680.0,France,0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,5.146,12680.0,France,0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,5.146,12680.0,France,0
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,6.138,12680.0,France,0


Now we have to preprocess our 'Description' column. We need to first change the case of this column to mixed case, because for example 'EARRING' contains the word 'RING' in it, and thus to avoid any confusion of jewelery item types, we should change the case to mixed so that the word starts with a single uppercase letter.

In [9]:
df['Description'] = df['Description'].str.title()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Target
0,536365,85123A,White Hanging Heart T-Light Holder,6,2010-12-01 08:26:00,3.1620,17850.0,United Kingdom,1
1,536365,71053,White Metal Lantern,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom,1
2,536365,84406B,Cream Cupid Hearts Coat Hanger,8,2010-12-01 08:26:00,3.4100,17850.0,United Kingdom,1
3,536365,84029G,Knitted Union Flag Hot Water Bottle,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom,1
4,536365,84029E,Red Woolly Hottie White Heart.,6,2010-12-01 08:26:00,4.2036,17850.0,United Kingdom,1


A quick look at the dataframe in Excel showed me that while every stock code seems unique, there is a specific code called 'POST' that is for postal service charges/shipping. I want to get rid of each row that contains 'POST' as a stockcode. 

In [10]:
# Want to find the count of 'POST' stockcodes before deleting them
print(df['StockCode'].str.contains(r'\bPOST\b', regex=True).sum())
print('Number of Rows: {}'.format(df.shape[0]))

1256
Number of Rows: 541909


In [11]:
# delete the rows with POST and count how many instances of rows there are after the deletion

# had an error at first: made sure the stockcode was a string type and set the argument na=false
df['StockCode'] = df['StockCode'].astype(str)

df = df[~df['StockCode'].str.contains(r'\bPOST\b', regex=True, na=False)]
print('Number of Rows: {}'.format(df.shape[0]))

Number of Rows: 540653


Now I need to do the preprocessing that relates to jewelery. I will have to pull out the possible jewelery words from the 'Description' column and make a new column of a binary classification which I will call 'Jewlery_Present', and 1 means yes, this item contains at least one word of 'jewlery' in it, or 0 where no, this item is not categorized as a jewlery item. 

In [12]:
# need to find all possible words that relate to jewlery first

# we set the expand argument inside of our split() to true so that it doesnt just return a list of strings
print(df['Description'].str.split(expand=True).stack().value_counts())

Set          54598
Of           53351
Bag          51911
Red          42902
Heart        38714
             ...  
Set?             1
Trading          1
Zero             1
Invc             1
Fragrance        1
Name: count, Length: 2536, dtype: int64


While my above method technically worked, it's not actually very optimal: it is clear that the number of unique words is too much to display all at once (~2500 words), but I need to be able to scroll through all of them. 

I will be borrowing a method from my data visualization class (IS 445) using a library called ipywidgets. ipywidgets is a library that can be used in environments such as Jupyter Notebooks to visualize python objects. Different types of widgets include things like sliders, toggle buttons, dropdown menus, etc. 

Documentation can be found at: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
This documentation also discusses IPython, which can be used in browsers like Jupyter Notebooks for integrating mixed media: https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html

In [13]:
import ipywidgets
from IPython.display import display

In [14]:
words = df['Description'].str.split(expand=True).stack().value_counts()

@ipywidgets.interact()
def display_words(rows=(5,60,5)):
    display(words.head(rows))

interactive(children=(IntSlider(value=30, description='rows', max=60, min=5, step=5), Output()), _dom_classes=…

While this method can be refined, it may be a bit more difficult than I originally thought it to be, and thus I would rather write a csv file to manually look through the words myself.